In [2]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('RDD Assignment').getOrCreate() # Creating Spark session object

In [ ]:
type(spark)

pyspark.sql.session.SparkSession

#### Question 1 : Create RDDs in three different ways.

In [ ]:
# Parallelize method :
par_rdd = spark.sparkContext.parallelize(['hello','world','how','are','you ?','All is well'])
par_rdd.collect()
par_rdd.count()

6

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# RDD from Data source :
ds_rdd = spark.sparkContext.textFile('input.txt')
ds_rdd.collect()
#ds_rdd.count()

['Harish is in ABD class.', 'Class is going on in ground floor']

In [ ]:
# Create RDD using transformations :
trans_rdd = ds_rdd.flatMap(lambda data:data.split(' '))
trans_rdd.collect()
trans_rdd.count()

12

#### Question 2 : Read a text file and count the number of words in the file using RDD operations.

In [ ]:
ds_rdd = spark.sparkContext.textFile('input.txt')
trans_rdd = ds_rdd.flatMap(lambda data:data.split(' '))
trans_rdd.collect()
trans_rdd.count()

12

####  Question 3 : Write a program to find the word frequency in a given file.

In [ ]:
trans_rdd.collect()
word_rdd = trans_rdd.map(lambda data: (data,1)) # creating a tuple
word1_rdd = word_rdd.reduceByKey(lambda a,b : a+b)
word1_rdd.collect()
#word1_rdd.count() # answer is 10

[('Harish', 1),
 ('is', 2),
 ('in', 2),
 ('ABD', 1),
 ('Class', 1),
 ('ground', 1),
 ('floor', 1),
 ('class.', 1),
 ('going', 1),
 ('on', 1)]

#### Question 4 : Write a program to convert all words in a file to uppercase.

In [ ]:
ds_rdd = spark.sparkContext.textFile('input.txt')
upper_rdd = ds_rdd.flatMap(lambda data:data.split(' '))
upper_rdd.collect()
# def convert_to_upper(data) :
#     if (data.isupper()) :
#         return (data)
#     else :
#         return (data.upper())
# upper1_rdd = upper_rdd.map(lambda data : convert_to_upper(data))
upper1_rdd = upper_rdd.map(lambda data : data.upper())
upper1_rdd.collect()

['HARISH',
 'IS',
 'IN',
 'ABD',
 'CLASS.',
 'CLASS',
 'IS',
 'GOING',
 'ON',
 'IN',
 'GROUND',
 'FLOOR']

#### Question 5 : Write a program to convert all words in a file to lowercase.

In [ ]:
lower_rdd = upper_rdd.map(lambda data : data.lower())
lower_rdd.collect()

['harish',
 'is',
 'in',
 'abd',
 'class.',
 'class',
 'is',
 'going',
 'on',
 'in',
 'ground',
 'floor']

#### Question 6 : Write a program to capitalize first letter of each words in file (use string capitalize() method).

In [ ]:
'''The capitalize() method returns a copy of the original string and converts the first character 
   of the string to a capital (uppercase) letter while making all other characters in the string lowercase letters.'''
capital_rdd = upper1_rdd.map(lambda data : data.capitalize())
capital_rdd.collect()

['Harish',
 'Is',
 'In',
 'Abd',
 'Class.',
 'Class',
 'Is',
 'Going',
 'On',
 'In',
 'Ground',
 'Floor.']

#### Question 7 : Find the longest length of word from given set of words.

In [ ]:
'''Harish is in ABD class.
Class is going on in ground floor.
Manipal is beautiful today.'''

long_rdd = spark.sparkContext.textFile('input.txt')
long1_rdd = long_rdd.flatMap(lambda data:data.split(' ')) \
            .map(lambda data : len(data))
#long1_rdd.collect()
#type(long1_rdd)
#type(long1_rdd.collect())
max(long1_rdd.collect())
# RDD is not a list, but RDD.collect() is a list.
# So max(long1_rdd) gives error but not max(long1_rdd.collect())

9

#### Question 8 : Map the Registration numbers to corresponding branch. 6000 series BDA, 9000 series HDA, 1000 series ML, 2000 series VLSI, 3000 series ES, 4000 series MSc, 5000 series CC. Given registration number, generate a key-value pair of Registration Number and Corresponding Branch.

In [ ]:
data_rdd = spark.sparkContext.textFile('D:/input.txt')
reg_rdd = data_rdd.flatMap(lambda data : data.split('\n'))
reg_rdd.collect()
# reg_rdd.count()  # 35

def branch(data):
    if(data[0:4] == '1000') :
        return 'ML'
    elif(data[0:4] == '2000') :
        return 'VLSI'
    elif(data[0:4] == '3000') :
        return 'ES'
    elif(data[0:4] == '4000') :
        return 'MSc'
    elif(data[0:4] == '5000') :
        return 'CC'
    elif(data[0:4] == '6000') :
        return 'BDA'
    elif(data[0:4] == '9000') :
        return 'HDA'

# Mapping registration number to its respective branch
reg1_rdd = reg_rdd.map(lambda data : (data,branch(data)))
reg1_rdd.collect()  
# To find count in each branch
reg1_rdd.map(lambda d : (d[1],1)).reduceByKey(lambda a,b : a+b).collect()

[('ES', 5),
 ('MSc', 5),
 ('CC', 5),
 ('BDA', 5),
 ('ML', 5),
 ('VLSI', 5),
 ('HDA', 5)]

#### Question 9 : Text file contain numbers. Numbers are separated by one white space. There is no order to store the numbers. One line may contain one or more numbers. Find the maximum, minimum, sum and mean of numbers.

In [ ]:
num_rdd = spark.sparkContext.textFile('C:/Users/MSIS/Desktop/numbers.txt')
num_rdd = num_rdd.flatMap(lambda data: data.split(' '))
num_rdd.collect()
max(num_rdd.map(lambda data : int(data)).collect())
#num_rdd.map(lambda data : int(data)).max()
min(num_rdd.map(lambda data : int(data)).collect())
#num_rdd.map(lambda data : int(data)).min()
sum(num_rdd.map(lambda data : int(data)).collect())
#num_rdd.map(lambda data : int(data)).sum()
num_rdd.map(lambda data : int(data)).mean()
#num_rdd.flatMap(lambda data: data.split(' ')).map(lambda data : int(data)).mean()

269.42105263157896

#### Question 10 : A text file (citizen.txt) contains data about citizens of country. Fields (information in file) are Name, dob, Phone, email and state name. Another file contains mapping of state names to state code like Karnataka is codes as KA, TamilNadu as TN, Kerala KL etc. Compress the citizen.txt file by changing full state name to state code. Write compressed data back into new file.

In [ ]:
# Creating RDD for the file which contains state name and its respective code :
# State code mapping
# This RDD should be broadcasted to all the systems doing the mapping of citizen_rdd.
state_rdd = spark.sparkContext.textFile('state_mapping.txt')\
        .map(lambda d:d.split(' '))
state_rdd.collect()

[['Karnataka', 'KA'],
 ['Himachal_Pradesh', 'HP'],
 ['Delhi', 'DL'],
 ['Maharashtra', 'MH'],
 ['Tamil_Nadu', 'TN'],
 ['West_Bengal', 'WB'],
 ['Rajasthan', 'RJ']]

In [ ]:
state_dict = {} # Creating empty dictionary
# state_dict is of dict type
for ele in state_rdd.collect():
    state_dict[ele[0]]=ele[1]  # Assigning values to key (Value is KA and key is Karnataka)
state_code = spark.sparkContext.broadcast(state_dict)
# state_code should be of broadcast type
#type(state_dict)  : Dict type
#type(state_code) : broadcast type
state_code
state_dict

{'Delhi': 'DL',
 'Himachal_Pradesh': 'HP',
 'Karnataka': 'KA',
 'Maharashtra': 'MH',
 'Rajasthan': 'RJ',
 'Tamil_Nadu': 'TN',
 'West_Bengal': 'WB'}

In [ ]:
# Original dataset
citizen_rdd = spark.sparkContext.textFile('citizen.txt')
citizen_rdd.collect()

['Harish R Sharma 02-01-1997 9845724355 hrs@gmail.com Karnataka',
 'Girish Shetty 03-08-2001 2348910233 gir123@gmail.com Maharashtra',
 'Mahesh Kumar 12-10-1978 0192384718 sdklfj@dllk.co.uk Delhi',
 'Satish Bhandary U A 31-9-2000 9871234811 dlkfsj@k;dlf.com Himachal_Pradesh',
 'Navami Patel 21-3-96 3928479823 navanavanavami@navanami.msnsjd Tamil_Nadu',
 'Greeshma Unnikrishnan M 4-06-2004 9871237865 lksjdffdhj@jdhfad.yg West_Bengal',
 'Pooja Gupta 19-05-1995 8746387912 jkhfdaf@kjad.com Rajasthan']

In [ ]:
# Passing entire row of a file into a function to do splitting and necessary overwriting
def map_state_code(data):
    data = data.split(' ')
    data[-1] = state_code.value.get(data[-1]) # last column is of interest here
    new_data = ','
    new_data = new_data.join(data)
    #new_data = ' '.join(data)
    return new_data

new_citizen = citizen_rdd.map(lambda d:map_state_code(d)) # State name is replaced by state code
new_citizen.collect()

['Harish,R,Sharma,02-01-1997,9845724355,hrs@gmail.com,KA',
 'Girish,Shetty,03-08-2001,2348910233,gir123@gmail.com,MH',
 'Mahesh,Kumar,12-10-1978,0192384718,sdklfj@dllk.co.uk,DL',
 'Satish,Bhandary,U,A,31-9-2000,9871234811,dlkfsj@k;dlf.com,HP',
 'Navami,Patel,21-3-96,3928479823,navanavanavami@navanami.msnsjd,TN',
 'Greeshma,Unnikrishnan,M,4-06-2004,9871237865,lksjdffdhj@jdhfad.yg,WB',
 'Pooja,Gupta,19-05-1995,8746387912,jkhfdaf@kjad.com,RJ']

In [ ]:
# Writing back the modified output to a new folder (Output inside folder will be in part files)
# Only one part file because repartition(1)
new_citizen.repartition(1).saveAsTextFile('C:/Users/MSIS/BDA21-SparkExamples/replaced_file')

In [ ]:
type(state_dict)
type(state_code) # State_code is of broadcast type

pyspark.broadcast.Broadcast

##### Question 11 : Create dataset (text file) with fields like ‘Student Name’, ‘Institute’, ‘Program Name’, and ‘Gender’ and solve following questions.
##### 1. Compute number of students from each Institute.
##### 2. Number of students enrolled to any program.
##### 3. Number of ‘boy’ and ‘girl’ students.
##### 4. Number of ‘boy’ and ‘girl’ students from selected Institute.

In [ ]:
'''Harish MSIS BDA Male
Prathik MIT ECE Male
Lavanya MSC Chemistry Female
Prathima MIT CSE Female
Kavvya MSIS AIML Female
Girish SMK Physics Male'''

student_rdd = spark.sparkContext.textFile('student.txt')
student_rdd.collect()

student1_rdd = student_rdd.map(lambda d:d.split(' ')).map(lambda d:(d[1],1)).reduceByKey(lambda a,b:a+b)
student1_rdd.collect()

student2_rdd = student_rdd.map(lambda d:d.split(' ')).map(lambda d:(d[2],1)).reduceByKey(lambda a,b:a+b)
student2_rdd.collect()

student3_rdd = student_rdd.map(lambda d:d.split(' ')).map(lambda d:(d[3],1)).reduceByKey(lambda a,b:a+b)
student3_rdd.collect()

student4_rdd = student_rdd.map(lambda d:d.split(' ')).map(lambda d:(d[3],1) if(d[1]=='SMK') else (d[3],0)).reduceByKey(lambda a,b:a+b)
student4_rdd.collect()

[('Male', 1), ('Female', 0)]

##### Question 12 : 
##### Dataset: Temperature of Indian Cities. Fields of dataset are Date, Average Temperature, City, Country, Latitude and Longitude (Use dataset attached to MapReduce assignment). Solve following questions
##### 1. Find maximum and minimum temperature of all cities from the given dataset
##### 2. Count number of data point for each city.
##### 3. Find the maximum and minimum temperature for city Bangalore from the given dataset.
##### 4. Find the maximum and minimum temperature for any given city from the given dataset. City name should be passed through command line argument. 

In [ ]:
temp_rdd = spark.sparkContext.textFile('sampleTemp.txt').map(lambda d:d.split(' '))
#temp_rdd.collect()

# Max and Min :
temp_max = temp_rdd.map(lambda d:(d[2],d[1])).reduceByKey(lambda a,b : max(a,b))
temp_max.collect()
temp_min = temp_rdd.map(lambda d:(d[2],d[1])).reduceByKey(lambda a,b : min(a,b))
temp_min.collect()

# Count :
count_city = temp_rdd.map(lambda d:(d[2],1)).reduceByKey(lambda a,b : a+b)
count_city.collect()

# Max and Min for Bangalore :
blore_max = temp_rdd.map(lambda d:(d[2],d[1])).filter(lambda d:(d[0]=="Bangalore")).reduceByKey(lambda a,b : max(a,b))
print(blore_max.collect())
blore_min = temp_rdd.map(lambda d:(d[2],d[1])).filter(lambda d:(d[0]=="Bangalore")).reduceByKey(lambda a,b : min(a,b))
print(blore_min.collect())
  

[('Bangalore', '29.688000000000002')]
[('Bangalore', '20.257')]


##### Question 13 : Create dataset (text file) of bank transactions. Fields in file are ‘Bank ID’, ‘Account Number’, ‘Transaction Date’, ‘Transaction Type’ (credit or debit), ‘Transaction Amount’. Date format is dd-mm-yyyy.
##### 1. Count unique number of customers
##### 2. Count unique number of Bank ID
##### 3. Count unique number of customers per Bank ID
##### 4. Number of transactions for given Account Number
##### 5. Number of credit transactions for given Account Number in a given year

In [7]:
'''
1001 211058027 21-06-2011 Cr 1012.23
2012 211012078 23-11-1978 Dr 20123.34
1399 132907341 11-03-2020 Cr 291.22
9876 093092223 09-12-1918 Dr 93180
1096 211058027 11-06-2000 Cr 1093.96
2012 132907341 14-06-2002 Cr 21191.22
9876 093092223 09-12-2018 Dr 93.913
'''
bank_data = spark.sparkContext.textFile('bank.txt').map(lambda d:d.split('\t'))
#columns = "`Bank_ID` STRING, `Account_Number` STRING, `Transaction_Date` STRING, `Transaction_Type` STRING, `Amount` STRING"
#bankDF = spark.createDataFrame(bank_data,columns)

In [12]:
# bankDF.show()
# bankDF.printSchema()
#bank_data.collect()

In [15]:
# Count unique number of customers
# bankDF.select('Bank_ID','Account_Number').groupBy('Bank_ID').count().collect() (Wrong approach)

bank_data1 = bank_data.map(lambda d:(d[1],1)).reduceByKey(lambda a,b:a+b)
bank_data1.collect()
#bank_data1.count()

[('123876122', 4),
 ('123872822', 2),
 ('123876171', 2),
 ('123876142', 4),
 ('123823122', 1),
 ('123809822', 1),
 ('123376422', 1),
 ('123802671', 1),
 ('123823822', 2),
 ('123872322', 3),
 ('123872829', 6),
 ('720127541', 2),
 ('123456789', 5),
 ('123822322', 1),
 ('123830822', 1)]

In [16]:
# Count unique number of Bank ID :
bank_data2 = bank_data.map(lambda d:(d[0],1)).reduceByKey(lambda a,b:a+b)
bank_data2.collect()
#bank_data2.count()

[('CAN00123', 12), ('SBI00042', 18), ('ICI00072', 6)]

------------------------------------------------------------------------
------------------------------------------------------------------------

In [62]:
# bank_data_3 = bankDF.select('Bank_ID','Account_Number').groupBy('Bank_ID').count()
# bank_data_3.show()

In [63]:
# bank_data_3.collect()

In [75]:
# Count unique number of customers per bank ID :
# bank_data3 = bank_data.map(lambda d:(d[0],d[1]))
# bank1 = bank_data3.toDF()

In [82]:
# bank1.show()
# bank1.createOrReplaceTempView("employee")  # Temporary view of filtered dataframe having name as 'employee'
# sqlDF = spark.sql("SELECT * FROM employee")
# sqlDF.orderBy('_1').show()


+----+---------+
|  _1|       _2|
+----+---------+
|1001|211058027|
|2012|211012078|
|1399|132907341|
|9876|093092223|
|1096|211058027|
|2012|132907341|
|9876|093092223|
+----+---------+

+----+---------+
|  _1|       _2|
+----+---------+
|1001|211058027|
|1096|211058027|
|1399|132907341|
|2012|132907341|
|2012|211012078|
|9876|093092223|
|9876|093092223|
+----+---------+



------------------------------------------------------------------------
------------------------------------------------------------------------

In [26]:
# Count unique number of customers per bank ID
bank_data.map(lambda data: (data[0],data[1])).groupByKey().map(lambda tup: (tup[0],set(tup[1]))).collect()

[('CAN00123',
  {'123456789',
   '123823822',
   '123872822',
   '123876122',
   '123876171',
   '720127541'}),
 ('SBI00042',
  {'123823822',
   '123872322',
   '123872822',
   '123872829',
   '123876122',
   '123876142',
   '123876171'}),
 ('ICI00072',
  {'123376422',
   '123802671',
   '123809822',
   '123822322',
   '123823122',
   '123830822'})]

In [28]:
bank_data3 = bank_data.map(lambda data: (data[0],data[1])).groupByKey().map(lambda tup: (tup[0],len(set(tup[1]))))
bank_data3.collect()

[('CAN00123', 6), ('SBI00042', 7), ('ICI00072', 6)]

In [29]:
# Number of transactions for given Account Number
bank_data4 = bank_data.map(lambda d:(d[1],1)).reduceByKey(lambda a,b : a+b)
bank_data4.collect()

[('123876122', 4),
 ('123872822', 2),
 ('123876171', 2),
 ('123876142', 4),
 ('123823122', 1),
 ('123809822', 1),
 ('123376422', 1),
 ('123802671', 1),
 ('123823822', 2),
 ('123872322', 3),
 ('123872829', 6),
 ('720127541', 2),
 ('123456789', 5),
 ('123822322', 1),
 ('123830822', 1)]

In [52]:
# Number of Credit transactions for given account number in a given year
#bank_data.map(lambda d:(d[1],d[2],d[3])).collect()
bank_data.map(lambda d:(d[1],d[2].split('-')[2],d[3])).filter(lambda d : (d[0]=="123876171")).collect()
#.map(lambda d : (d[2],1).filter.collect()

[('123876171', '2021', 'D'), ('123876171', '2005', 'D')]

In [53]:
bank_data.map(lambda d:(d[1],d[2].split('-')[2],d[3])).filter(lambda d : ((d[0]=="123876171") & (d[1]=="2021") & (d[2]=="C"))).count()

0